In [3]:
from legacy.possession import *
from sklearn.externals import joblib
import numpy as np
import pandas as pd
import glob
import tqdm
from collections import Counter
import seaborn as sns
from legacy.code_mapping import gid2name, code2team

In [4]:
p = joblib.load('Possession(2017012001, 1v4, 603.5-558.9, q2, Field Goal Missed, p0).pkl')

In [5]:
p.animate()
plt.show()

/home/hzwang/anaconda3/envs/probabilistic/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


ValueError: Invalid file object: <_io.BufferedReader name=60>